# Machine Learning in Python - MNIST Fashion

## Data Prep

### Lots of Imports

We need to Upload the our credentials of the Kaggle account in here befoew we could import any dataset. 


1.   Log into you Kaggle acount
2.   Click on the "your profile" icon on the top right of the page
3.   Click on My Account
4.   Below the page click on Create New API Token

The Kaggle.json file will be downloaded.

Now you can run the code block down below and upload the Kaggle.json file from your PC onto Google Colab. This way you can automatically loginto your Kaggle account with no problem and seamlessly.



In [0]:
# Upload the credentials of the Kaggle account
from google.colab import files
files.upload() 

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mehrandragon","key":"251e5b5a01ce62e6017d8d4d64e7786e"}'}

In [2]:
# Before importing the datasets we need to run the following code
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In order to download a dataset from Kaggle, we need the API Command for that dataset. to get it, choose the dataset of your interest on Kaggle, then choose "**Copy API Command**" and paste it here RIGHT after the exclamation mark down below . For example, for the  Fashion MNIST dataset the API command is: 
***kaggle datasets download -d zalando-research/fashionmnist***

In [3]:
# Now we can import the dataset of interest
!kaggle datasets download -d zalando-research/fashionmnist

 16%|██████▏                                | 11.0M/68.8M [00:07<01:26, 698kB/s]/Users/bmacnamee/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|███████████████████████████████████████| 68.8M/68.8M [03:13<00:00, 348kB/s]



In [1]:
# We can see the imnported files from Kaggle
!ls

X_test.csv                  sample_submission.csv
X_train.csv                 sub1.csv
kaggle_career_con_nb1.ipynb y_train.csv


In [2]:
import os
import subprocess
from IPython.display import display, HTML, Image
import io
from operator import itemgetter

# from TAS_Python_Utilities import data_viz
# from TAS_Python_Utilities import data_viz_target
# from TAS_Python_Utilities import visualize_tree

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
from random import randint
from scipy.misc import toimage

from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn import metrics
from sklearn import tree
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import linear_model
from sklearn import neighbors
from sklearn import neural_network

%matplotlib inline
#%qtconsole

To build predictive models in Python we use a set of libraries that are imported here. In particular **pandas** and **sklearn** are particularly important.

### Setup - IMPORTANT

Take only a sample of the dataset for fast testing

In [4]:
data_sampling_rate = 1.0

Setup the number of folds for all grid searches (should be 5 - 10)

In [5]:
cv_folds =  5

Set up a dictionary to store simple model perofrmance comparions

In [8]:
model_valid_accuracy_comparisons = dict()
model_tuned_params_list = dict()

### Load & Partition Data

Load the dataset and explore it.

In [6]:
x_train_dataset = pd.read_csv('X_train.csv')
#x_train_dataset = x_train_dataset.sample(frac=data_sampling_rate) #take a sample from the dataset so everyhting runs smoothly

display(x_train_dataset.head())
x_train_dataset.shape

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410


(487680, 13)

In [7]:
x_test_dataset = pd.read_csv('X_test.csv')
#x_train_dataset = x_train_dataset.sample(frac=data_sampling_rate) #take a sample from the dataset so everyhting runs smoothly

display(x_test_dataset.head())
x_test_dataset.shape

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.025773,-0.98864,-0.14801,0.003350,-0.006524,-0.001071,-0.027390,0.10043,4.2061,-5.5439
1,0_1,0,1,-0.025683,-0.98862,-0.14816,0.003439,-0.113960,0.083987,-0.060590,-0.70889,3.9905,-8.0273
2,0_2,0,2,-0.025617,-0.98861,-0.14826,0.003571,-0.080518,0.114860,-0.037177,1.45710,2.2828,-11.2990
3,0_3,0,3,-0.025566,-0.98862,-0.14817,0.003609,0.070067,0.033820,-0.035904,0.71096,1.8582,-12.2270
4,0_4,0,4,-0.025548,-0.98866,-0.14792,0.003477,0.152050,-0.029016,-0.015314,3.39960,2.7881,-10.4100


(488448, 13)

In [8]:
print(x_train_dataset['series_id'].nunique())

test_df = x_train_dataset.groupby(['series_id'])['orientation_X', 'orientation_Y', 'orientation_Z', 'orientation_W', \
                                                 'angular_velocity_X', 'angular_velocity_Y', 'angular_velocity_Z',  \
                                                'linear_acceleration_X', 'linear_acceleration_Y', 'linear_acceleration_Z'].mean().reset_index()

#test_df = test_df.set_index('series_id')
test_df_norm = (test_df - test_df.mean()) / (test_df.max() - test_df.min())
test_df_norm['series_id'] = test_df['series_id']
display(test_df_norm.head())
print(test_df_norm.shape)

3810


,series_id,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0,-0.374524,-0.358449,-0.378657,-0.332880,-0.008415,-0.019019,0.013534,0.059338,0.034329,0.029527
1,1,-0.475632,0.084323,0.061624,-0.463890,0.014014,-0.026289,0.012863,-0.003279,-0.041546,-0.015935
2,2,-0.249816,-0.465702,-0.455384,-0.218140,0.007813,-0.028698,0.023899,0.009038,0.000097,-0.020503
3,3,-0.465803,0.118837,0.085078,-0.449116,0.001413,-0.018199,0.013714,0.032076,-0.080281,-0.057254
4,4,-0.441596,0.178490,0.152831,-0.430120,0.021498,0.061696,-0.062416,-0.095308,0.033381,0.009886


(3810, 11)


In [9]:
print(x_test_dataset['series_id'].nunique())

test_df2 = x_test_dataset.groupby(['series_id'])['orientation_X', 'orientation_Y', 'orientation_Z', 'orientation_W', \
                                                 'angular_velocity_X', 'angular_velocity_Y', 'angular_velocity_Z',  \
                                                'linear_acceleration_X', 'linear_acceleration_Y', 'linear_acceleration_Z'].mean().reset_index()

#test_df = test_df.set_index('series_id')
test_df_norm2 = (test_df2 - test_df2.mean()) / (test_df2.max() - test_df2.min())
test_df_norm2['series_id'] = test_df2['series_id']
display(test_df_norm2.head())
print(test_df_norm2.shape)

3816


,series_id,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0,-0.029214,-0.560780,-0.544746,-0.002856,0.002721,-0.016565,0.015286,0.006670,-0.006875,0.029214
1,1,-0.487327,0.105968,0.080633,-0.472482,0.046778,0.001434,-0.000368,-0.042447,0.003718,0.011910
2,2,-0.132501,0.425046,0.410354,-0.151260,-0.017816,-0.006887,0.004381,0.040207,0.004256,-0.081689
3,3,0.067046,0.432045,0.416275,0.045858,0.017516,-0.030737,0.028779,0.016182,0.013839,0.001728
4,4,-0.144333,0.422146,0.403773,-0.156713,0.022369,0.083831,-0.080910,-0.026421,-0.037434,-0.030977


(3816, 11)


Examine the distribution of the two classes

In [11]:
num_classes = 9
classes = {0: "fine_concrete", 1:"concrete", 2: "soft_tiles", 3:"tiled", 4:"soft_pvc", 5:"hard_tiles_large_space", 6:"carpet", 7:"hard_tiles", 8:"wood"}

y_train_dataset  = pd.read_csv('y_train.csv')
display(y_train_dataset.head())

#series_ids = list(set(x_train_dataset['series_id']))
#print(series_ids)

#y_train_dataset = y_train_load
#y_train_dataset = y_train_dataset[y_train_load['series_id'].isin(series_ids)]
y_train_dataset.shape

,series_id,group_id,surface
0,0,13,fine_concrete
1,1,31,concrete
2,2,20,concrete
3,3,31,concrete
4,4,22,soft_tiles


(3810, 3)

In [12]:
# Check for presence of missing values
print("Missing Values")
print(x_train_dataset.isnull().sum())
print(y_train_dataset.isnull().sum())

print(x_test_dataset.isnull().sum())

Missing Values
row_id                   0
series_id                0
measurement_number       0
orientation_X            0
orientation_Y            0
orientation_Z            0
orientation_W            0
angular_velocity_X       0
angular_velocity_Y       0
angular_velocity_Z       0
linear_acceleration_X    0
linear_acceleration_Y    0
linear_acceleration_Z    0
dtype: int64
series_id    0
group_id     0
surface      0
dtype: int64
row_id                   0
series_id                0
measurement_number       0
orientation_X            0
orientation_Y            0
orientation_Z            0
orientation_W            0
angular_velocity_X       0
angular_velocity_Y       0
angular_velocity_Z       0
linear_acceleration_X    0
linear_acceleration_Y    0
linear_acceleration_Z    0
dtype: int64


In [0]:
# Visualise fields
#data_viz(dataset)

In [0]:
# Visualise fields in relation to target
#data_viz_target(dataset, "label")

Isolate the descriptive features we are interested in

In [13]:
X = test_df_norm[test_df_norm.columns[1:]]
display(X)
Y = np.array(y_train_dataset["surface"])
display(Y)

,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,-0.374524,-0.358449,-0.378657,-0.332880,-0.008415,-0.019019,0.013534,0.059338,0.034329,0.029527
1,-0.475632,0.084323,0.061624,-0.463890,0.014014,-0.026289,0.012863,-0.003279,-0.041546,-0.015935
2,-0.249816,-0.465702,-0.455384,-0.218140,0.007813,-0.028698,0.023899,0.009038,0.000097,-0.020503
3,-0.465803,0.118837,0.085078,-0.449116,0.001413,-0.018199,0.013714,0.032076,-0.080281,-0.057254
4,-0.441596,0.178490,0.152831,-0.430120,0.021498,0.061696,-0.062416,-0.095308,0.033381,0.009886
5,0.244129,-0.479010,-0.448621,0.275650,-0.023880,-0.013839,0.006233,-0.008568,-0.076409,0.052851
6,-0.194341,-0.494502,-0.491252,-0.155780,0.032418,0.015334,-0.016097,0.020300,-0.086384,-0.066448
7,-0.459097,0.137802,0.109548,-0.442251,0.028508,-0.013818,0.005600,0.140569,-0.101145,0.034230
8,0.015217,-0.537866,-0.513921,0.043603,-0.017343,-0.045504,0.037566,-0.064164,-0.147830,-0.098506
9,0.408188,-0.338355,-0.307171,0.421104,-0.024488,-0.382729,0.384359,0.187956,0.173938,0.093454


array(['fine_concrete', 'concrete', 'concrete', ..., 'fine_concrete',
       'tiled', 'soft_pvc'], dtype=object)

In [15]:
X_test = test_df_norm2[test_df_norm2.columns[1:]]
display(X_test)

,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,-0.029214,-0.560780,-0.544746,-0.002856,0.002721,-0.016565,0.015286,0.006670,-0.006875,0.029214
1,-0.487327,0.105968,0.080633,-0.472482,0.046778,0.001434,-0.000368,-0.042447,0.003718,0.011910
2,-0.132501,0.425046,0.410354,-0.151260,-0.017816,-0.006887,0.004381,0.040207,0.004256,-0.081689
3,0.067046,0.432045,0.416275,0.045858,0.017516,-0.030737,0.028779,0.016182,0.013839,0.001728
4,-0.144333,0.422146,0.403773,-0.156713,0.022369,0.083831,-0.080910,-0.026421,-0.037434,-0.030977
5,-0.292840,-0.477252,-0.480854,-0.260339,0.050158,-0.109751,0.060417,-0.317636,0.024710,0.138536
6,-0.421084,0.231490,0.216144,-0.426643,0.028238,-0.030527,0.041090,0.020214,0.034539,0.045282
7,-0.337955,0.321763,0.290751,-0.333600,-0.024490,0.049677,-0.041286,-0.032459,0.035880,-0.023877
8,-0.419260,0.234532,0.204635,-0.415931,0.026473,0.117355,-0.116460,-0.020643,-0.072895,-0.091007
9,0.197023,0.390985,0.391821,0.176154,-0.001146,-0.089465,0.093383,0.003612,0.003260,-0.006075


Display some of the images

Normalise the data (important for some models)

In [16]:
X = X/255

Split the data into a **training set**, a **vaidation set**, and a **test set**

In [16]:
# X_train_plus_valid, X_test, y_train_plus_valid, y_test \
#     = train_test_split(X, Y, random_state=0, \
#                                     train_size = 0.7)

X_train, X_valid, y_train, y_valid \
    = train_test_split(X, Y, \
                        random_state=0, \
                        train_size = 0.5/0.7)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


## Building Simple Models

### A Very Simple Decision Tree

Train a decision tree

In [70]:
my_tree = tree.DecisionTreeClassifier(criterion="entropy")
my_tree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Visualise the decision tree so we can see what it is doing!

In [19]:
#feature_names = list(X_train.columns)
#visualize_tree(my_tree, feature_names, fileName='dt_over.png')
#Image(filename='dt_over.png') 

### Evaluating Model Performance

Assess the performance of the decision tree on the training set

In [71]:
# Make a set of predictions for the training data
y_pred = my_tree.predict(X_train)

# Print performance details
accuracy = metrics.accuracy_score(y_train, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_train, y_pred))

# Print confusion matrix
# print(metrics.confusion_matrix(y_train, y_pred))

# Print nicer homemade confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_train), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 1.0
                        precision    recall  f1-score   support

                carpet       1.00      1.00      1.00        94
              concrete       1.00      1.00      1.00       399
         fine_concrete       1.00      1.00      1.00       173
            hard_tiles       1.00      1.00      1.00        10
hard_tiles_large_space       1.00      1.00      1.00       151
              soft_pvc       1.00      1.00      1.00       368
            soft_tiles       1.00      1.00      1.00       153
                 tiled       1.00      1.00      1.00       253
                  wood       1.00      1.00      1.00       304

           avg / total       1.00      1.00      1.00      1905

Confusion Matrix


Predicted,carpet,concrete,fine_concrete,hard_tiles,hard_tiles_large_space,soft_pvc,soft_tiles,tiled,wood,All
True,,,,,,,,,,
carpet,94,0,0,0,0,0,0,0,0,94
concrete,0,399,0,0,0,0,0,0,0,399
fine_concrete,0,0,173,0,0,0,0,0,0,173
hard_tiles,0,0,0,10,0,0,0,0,0,10
hard_tiles_large_space,0,0,0,0,151,0,0,0,0,151
soft_pvc,0,0,0,0,0,368,0,0,0,368
soft_tiles,0,0,0,0,0,0,153,0,0,153
tiled,0,0,0,0,0,0,0,253,0,253
wood,0,0,0,0,0,0,0,0,304,304


Assess the performance of the tree on the validation dataset

In [72]:
# Make a set of predictions for the validation data
y_pred = my_tree.predict(X_valid)

# Print performance details
accuracy = metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
model_valid_accuracy_comparisons["Simple Tree"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
# print(metrics.confusion_matrix(y_valid, y_pred))

# Print nicer confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_valid), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.6391076115485564
                        precision    recall  f1-score   support

                carpet       0.38      0.42      0.40        31
              concrete       0.69      0.61      0.65       158
         fine_concrete       0.67      0.62      0.64        84
            hard_tiles       0.25      0.25      0.25         4
hard_tiles_large_space       0.63      0.63      0.63        62
              soft_pvc       0.71      0.75      0.73       148
            soft_tiles       0.53      0.71      0.60        55
                 tiled       0.73      0.74      0.73       103
                  wood       0.54      0.50      0.52       117

           avg / total       0.64      0.64      0.64       762

Confusion Matrix


Predicted,carpet,concrete,fine_concrete,hard_tiles,hard_tiles_large_space,soft_pvc,soft_tiles,tiled,wood,All
True,,,,,,,,,,
carpet,13,4,2,0,0,2,3,3,4,31
concrete,4,97,3,1,11,11,7,11,13,158
fine_concrete,3,8,52,0,1,6,4,1,9,84
hard_tiles,0,0,1,1,0,0,1,0,1,4
hard_tiles_large_space,2,6,2,0,39,4,1,1,7,62
soft_pvc,0,6,3,0,1,111,7,6,14,148
soft_tiles,0,5,3,0,1,2,39,4,1,55
tiled,4,2,3,1,1,10,5,76,1,103
wood,8,12,9,1,8,11,7,2,59,117


Assess the performance of the tree on the validation dataset

In [73]:
# Make a set of predictions for the test data
y_pred = my_tree.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
model_test_accuracy_comparisons["Simple Tree"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
# print(metrics.confusion_matrix(y_test, y_pred))

# Print nicer confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_test), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.6395450568678915
                        precision    recall  f1-score   support

                carpet       0.45      0.42      0.44        64
              concrete       0.65      0.60      0.63       222
         fine_concrete       0.64      0.62      0.63       106
            hard_tiles       0.75      0.43      0.55         7
hard_tiles_large_space       0.70      0.65      0.68        95
              soft_pvc       0.69      0.76      0.73       216
            soft_tiles       0.52      0.67      0.59        89
                 tiled       0.75      0.68      0.71       158
                  wood       0.57      0.58      0.58       186

           avg / total       0.64      0.64      0.64      1143

Confusion Matrix


Predicted,carpet,concrete,fine_concrete,hard_tiles,hard_tiles_large_space,soft_pvc,soft_tiles,tiled,wood,All
True,,,,,,,,,,
carpet,27,14,2,0,1,1,5,4,10,64
concrete,7,134,5,0,9,22,17,5,23,222
fine_concrete,1,9,66,1,3,9,3,3,11,106
hard_tiles,0,0,0,3,1,0,0,0,3,7
hard_tiles_large_space,3,5,2,0,62,9,4,4,6,95
soft_pvc,0,13,7,0,3,164,4,10,15,216
soft_tiles,2,8,5,0,1,2,60,6,5,89
tiled,10,8,5,0,0,7,14,107,7,158
wood,10,15,11,0,8,22,8,4,108,186


### Less Overiftted Decision Tree

Train a decision tree, setting min samples per leaf to a sensible value

In [74]:
my_tree = tree.DecisionTreeClassifier(criterion="entropy", min_samples_split = 200)
my_tree = my_tree.fit(X_train,y_train)

Visualise the decision tree so we can see what it is doing!

In [24]:
# visualise the decision tree
#feature_names = list(X_train.columns)
#visualize_tree(my_tree, feature_names, fileName="dt_under.png")
#Image(filename='dt_under.png') 

Assess the performance of the decision tree on the **training set**

In [75]:
# Make a set of predictions for the training data
y_pred = my_tree.predict(X_train)

# Print performance details
accuracy = metrics.accuracy_score(y_train, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_train, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_train), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.48871391076115483
                        precision    recall  f1-score   support

                carpet       0.34      0.45      0.39        94
              concrete       0.48      0.71      0.58       399
         fine_concrete       0.34      0.18      0.24       173
            hard_tiles       0.00      0.00      0.00        10
hard_tiles_large_space       0.35      0.45      0.39       151
              soft_pvc       0.80      0.40      0.54       368
            soft_tiles       0.24      0.14      0.18       153
                 tiled       0.55      0.76      0.63       253
                  wood       0.51      0.47      0.49       304

           avg / total       0.51      0.49      0.47      1905

Confusion Matrix


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted,carpet,concrete,fine_concrete,hard_tiles_large_space,soft_pvc,soft_tiles,tiled,wood,All
True,,,,,,,,,
carpet,42,22,0,0,0,0,25,5,94
concrete,5,285,5,13,24,7,19,41,399
fine_concrete,18,40,31,11,0,18,27,28,173
hard_tiles,0,1,0,9,0,0,0,0,10
hard_tiles_large_space,31,14,4,68,0,9,2,23,151
soft_pvc,3,73,8,41,149,16,48,30,368
soft_tiles,10,66,29,9,0,22,10,7,153
tiled,3,33,2,7,1,14,192,1,253
wood,12,56,11,36,12,6,29,142,304


Assess the performance of the decision tree on the **validation set**

In [76]:
# Make a set of predictions for the test data
y_pred = my_tree.predict(X_valid)

# Print performance details
accuracy = metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
model_valid_accuracy_comparisons["Better Tree"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_valid), y_pred, rownames=['True'], colnames=['Predicted'], margins=True, dropna = False)

Accuracy: 0.46062992125984253
                        precision    recall  f1-score   support

                carpet       0.29      0.48      0.37        31
              concrete       0.42      0.62      0.50       158
         fine_concrete       0.29      0.15      0.20        84
            hard_tiles       0.00      0.00      0.00         4
hard_tiles_large_space       0.43      0.48      0.46        62
              soft_pvc       0.82      0.51      0.63       148
            soft_tiles       0.29      0.20      0.24        55
                 tiled       0.52      0.67      0.58       103
                  wood       0.38      0.34      0.36       117

           avg / total       0.48      0.46      0.45       762

Confusion Matrix


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted,carpet,concrete,fine_concrete,hard_tiles_large_space,soft_pvc,soft_tiles,tiled,wood,All
True,,,,,,,,,
carpet,15,10,0,0,0,0,2,4,31
concrete,1,98,4,8,8,2,7,30,158
fine_concrete,10,24,13,1,0,5,15,16,84
hard_tiles,0,1,0,3,0,0,0,0,4
hard_tiles_large_space,12,6,2,30,1,3,2,6,62
soft_pvc,0,31,4,7,75,6,20,5,148
soft_tiles,2,19,16,1,0,11,4,2,55
tiled,2,18,2,2,1,8,69,1,103
wood,9,24,4,17,6,3,14,40,117


In [77]:
# Make a set of predictions for the test data
y_pred = my_tree.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
model_test_accuracy_comparisons["Better Tree"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_test), y_pred, rownames=['True'], colnames=['Predicted'], margins=True, dropna = False)

Accuracy: 0.4558180227471566
                        precision    recall  f1-score   support

                carpet       0.29      0.38      0.33        64
              concrete       0.46      0.68      0.55       222
         fine_concrete       0.23      0.12      0.16       106
            hard_tiles       0.00      0.00      0.00         7
hard_tiles_large_space       0.35      0.44      0.39        95
              soft_pvc       0.79      0.46      0.58       216
            soft_tiles       0.19      0.12      0.15        89
                 tiled       0.56      0.72      0.63       158
                  wood       0.40      0.37      0.38       186

           avg / total       0.46      0.46      0.44      1143

Confusion Matrix


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted,carpet,concrete,fine_concrete,hard_tiles_large_space,soft_pvc,soft_tiles,tiled,wood,All
True,,,,,,,,,
carpet,24,20,0,0,0,0,12,8,64
concrete,2,150,3,7,8,5,8,39,222
fine_concrete,11,18,13,9,0,13,20,22,106
hard_tiles,0,0,0,7,0,0,0,0,7
hard_tiles_large_space,24,7,2,42,0,9,2,9,95
soft_pvc,3,41,3,18,100,6,26,19,216
soft_tiles,4,35,22,5,1,11,7,4,89
tiled,6,15,2,3,3,13,113,3,158
wood,8,38,11,30,14,2,15,68,186


## Choosing Parameters Using a Grid Search

Use a cross validation to perfrom an evaluation

In [28]:
my_tree = tree.DecisionTreeClassifier(max_depth = 12)
scores = cross_val_score(my_tree, X_train_plus_valid, y_train_plus_valid, cv=10)
print(scores)

[0.74823529 0.7464455  0.72921615 0.75       0.73095238 0.72619048
 0.74761905 0.73205742 0.73444976 0.76442308]


An alternative to using post pruning explicitly is to use a grid search through a large set of possible parameters. Here we try depths between 3 and 20 and different limits on the minimum number of samples per split.

In [29]:
# Set up the parameter grid to seaerch
param_grid ={'criterion': ['gini', "entropy"], \
             'max_depth': list(range(3, 50, 3)), \
             'min_samples_split': [200]}

# Perform the search
my_tuned_tree = GridSearchCV(tree.DecisionTreeClassifier(), \
                                param_grid, cv=cv_folds, verbose = 2, \
                            return_train_score=True)
my_tuned_tree.fit(X_train_plus_valid, y_train_plus_valid)

# Print details
print("Best parameters set found on development set:")
display(my_tuned_tree.best_params_)
model_tuned_params_list["Tuned Tree"] = my_tuned_tree.best_params_
display(my_tuned_tree.best_score_)
display(my_tuned_tree.cv_results_)

Fitting 2 folds for each of 32 candidates, totalling 64 fits
[CV] criterion=gini, max_depth=3, min_samples_split=200 ..............
[CV]  criterion=gini, max_depth=3, min_samples_split=200, total=   0.2s
[CV] criterion=gini, max_depth=3, min_samples_split=200 ..............
[CV]  criterion=gini, max_depth=3, min_samples_split=200, total=   0.2s
[CV] criterion=gini, max_depth=6, min_samples_split=200 ..............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  criterion=gini, max_depth=6, min_samples_split=200, total=   0.3s
[CV] criterion=gini, max_depth=6, min_samples_split=200 ..............
[CV]  criterion=gini, max_depth=6, min_samples_split=200, total=   0.3s
[CV] criterion=gini, max_depth=9, min_samples_split=200 ..............
[CV]  criterion=gini, max_depth=9, min_samples_split=200, total=   0.3s
[CV] criterion=gini, max_depth=9, min_samples_split=200 ..............
[CV]  criterion=gini, max_depth=9, min_samples_split=200, total=   0.3s
[CV] criterion=gini, max_depth=12, min_samples_split=200 .............
[CV]  criterion=gini, max_depth=12, min_samples_split=200, total=   0.3s
[CV] criterion=gini, max_depth=12, min_samples_split=200 .............
[CV]  criterion=gini, max_depth=12, min_samples_split=200, total=   0.3s
[CV] criterion=gini, max_depth=15, min_samples_split=200 .............
[CV]  criterion=gini, max_depth=15, min_samples_split=200, total=   0.3s
[CV] criterion=gini, max_depth=15, min_samples_split=200 ..........

[CV]  criterion=entropy, max_depth=42, min_samples_split=200, total=   0.5s
[CV] criterion=entropy, max_depth=45, min_samples_split=200 ..........
[CV]  criterion=entropy, max_depth=45, min_samples_split=200, total=   0.4s
[CV] criterion=entropy, max_depth=45, min_samples_split=200 ..........
[CV]  criterion=entropy, max_depth=45, min_samples_split=200, total=   0.5s
[CV] criterion=entropy, max_depth=48, min_samples_split=200 ..........
[CV]  criterion=entropy, max_depth=48, min_samples_split=200, total=   0.4s
[CV] criterion=entropy, max_depth=48, min_samples_split=200 ..........
[CV]  criterion=entropy, max_depth=48, min_samples_split=200, total=   0.5s


[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   23.9s finished


Best parameters set found on development set:


{'criterion': 'entropy', 'max_depth': 6, 'min_samples_split': 200}

0.6902380952380952

{'mean_fit_time': array([0.17427051, 0.29040933, 0.29170454, 0.3037529 , 0.27801788,
        0.27113903, 0.27169704, 0.26272297, 0.26308298, 0.26486301,
        0.2801199 , 0.28788519, 0.29952598, 0.28118944, 0.2838695 ,
        0.30352557, 0.2568351 , 0.46159637, 0.44768941, 0.45452452,
        0.45159161, 0.44803488, 0.44784856, 0.48169899, 0.48510253,
        0.48310244, 0.469733  , 0.46219254, 0.46866906, 0.46726251,
        0.46606326, 0.46651292]),
 'mean_score_time': array([0.00439465, 0.00368178, 0.00360608, 0.00450003, 0.00284398,
        0.00324094, 0.00424194, 0.00274861, 0.00283086, 0.00318491,
        0.00329006, 0.00430489, 0.00393355, 0.00527203, 0.00305355,
        0.0041759 , 0.00308335, 0.00309956, 0.00376606, 0.00308645,
        0.00292432, 0.00291407, 0.00436711, 0.00493491, 0.00388932,
        0.00421607, 0.00328803, 0.00309992, 0.004439  , 0.00462842,
        0.00329006, 0.00369823]),
 'mean_test_score': array([0.45380952, 0.68214286, 0.6802381 , 0.6802381 , 0.680

Evaluate the performance of the tuned tree

In [30]:
# Make a set of predictions for the test data
y_pred = my_tuned_tree.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
model_test_accuracy_comparisons["Tuned Tree"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_test), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.7455555555555555
             precision    recall  f1-score   support

          0       0.77      0.71      0.74       192
          1       0.98      0.83      0.90       185
          2       0.69      0.61      0.65       157
          3       0.54      0.89      0.67       188
          4       0.57      0.51      0.54       157
          5       0.79      0.74      0.76       170
          6       0.58      0.45      0.51       173
          7       0.81      0.86      0.84       181
          8       0.85      0.90      0.88       199
          9       0.93      0.86      0.89       198

avg / total       0.76      0.75      0.75      1800

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,137,0,4,33,2,1,2,0,13,0,192
1,1,153,6,23,1,1,0,0,0,0,185
2,0,0,96,5,39,0,14,0,3,0,157
3,8,2,5,167,1,1,1,0,3,0,188
4,1,1,11,33,80,0,31,0,0,0,157
5,0,0,2,11,0,125,0,25,3,4,170
6,32,0,15,26,14,0,78,0,8,0,173
7,0,0,0,1,0,16,0,156,0,8,181
8,0,0,1,4,3,3,7,1,180,0,199


Visualise the tree

In [31]:
best_tree = tree.DecisionTreeClassifier(min_samples_split=200, criterion='gini', max_depth=8)
best_tree = best_tree.fit(X_train, y_train)

# visualise the decision tree
#feature_names = list(X_train.columns)
#visualize_tree(best_tree, feature_names, 'dt_tuned.png')
#Image(filename='dt_tuned.png') 

## Comparing Models

We can easily use the same patterns to train other types of models.

#### Random Forests

Train and evaluate a simple model 

In [32]:
# Do the same job with random forests
my_model = ensemble.RandomForestClassifier(n_estimators=300, \
                                           max_features = 3,\
                                           min_samples_split=200)
my_model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=200,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [33]:
# Make a set of predictions for the test data
y_pred = my_model.predict(X_valid)

# Print performance details
accuracy = metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
model_valid_accuracy_comparisons["Random Forest"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_valid), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.7608333333333334
             precision    recall  f1-score   support

          0       0.72      0.84      0.78       128
          1       0.87      0.92      0.89       120
          2       0.56      0.75      0.64       100
          3       0.69      0.82      0.75       117
          4       0.64      0.60      0.62       139
          5       0.92      0.79      0.85       111
          6       0.64      0.23      0.33       124
          7       0.82      0.83      0.82       119
          8       0.91      0.93      0.92       124
          9       0.82      0.94      0.88       118

avg / total       0.76      0.76      0.75      1200

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,108,0,2,8,2,0,5,0,3,0,128
1,2,110,5,3,0,0,0,0,0,0,120
2,3,0,75,0,18,0,3,0,1,0,100
3,2,12,3,96,1,0,3,0,0,0,117
4,1,3,24,21,83,0,5,0,2,0,139
5,0,0,0,0,0,88,0,14,0,9,111
6,33,1,23,9,25,0,28,0,5,0,124
7,0,0,0,0,0,5,0,99,0,15,119
8,0,0,1,2,1,3,0,2,115,0,124


Choose parameters using a grid search

In [34]:
# Set up the parameter grid to seaerch
param_grid = [
 {'n_estimators': list(range(100, 501, 50)), 'max_features': list(range(2, 10, 2)), 'min_samples_split': [200] }
]

# Perform the search
my_tuned_model = GridSearchCV(ensemble.RandomForestClassifier(), param_grid, cv=cv_folds, verbose = 2)
my_tuned_model.fit(X_train_plus_valid, y_train_plus_valid)

# Print details
print("Best parameters set found on development set:")
print(my_tuned_model.best_params_)
model_tuned_params_list["Tuned Random Forest"] = my_tuned_model.best_params_
print(my_tuned_model.best_score_)

Fitting 2 folds for each of 36 candidates, totalling 72 fits
[CV] max_features=2, min_samples_split=200, n_estimators=100 .........
[CV]  max_features=2, min_samples_split=200, n_estimators=100, total=   0.2s
[CV] max_features=2, min_samples_split=200, n_estimators=100 .........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  max_features=2, min_samples_split=200, n_estimators=100, total=   0.2s
[CV] max_features=2, min_samples_split=200, n_estimators=150 .........
[CV]  max_features=2, min_samples_split=200, n_estimators=150, total=   0.3s
[CV] max_features=2, min_samples_split=200, n_estimators=150 .........
[CV]  max_features=2, min_samples_split=200, n_estimators=150, total=   0.3s
[CV] max_features=2, min_samples_split=200, n_estimators=200 .........
[CV]  max_features=2, min_samples_split=200, n_estimators=200, total=   0.4s
[CV] max_features=2, min_samples_split=200, n_estimators=200 .........
[CV]  max_features=2, min_samples_split=200, n_estimators=200, total=   0.4s
[CV] max_features=2, min_samples_split=200, n_estimators=250 .........
[CV]  max_features=2, min_samples_split=200, n_estimators=250, total=   0.5s
[CV] max_features=2, min_samples_split=200, n_estimators=250 .........
[CV]  max_features=2, min_samples_split=200, n_estimators=250, total=   0.5s
[CV] max_features=2, min_samples_sp

[CV]  max_features=8, min_samples_split=200, n_estimators=150, total=   0.4s
[CV] max_features=8, min_samples_split=200, n_estimators=200 .........
[CV]  max_features=8, min_samples_split=200, n_estimators=200, total=   0.5s
[CV] max_features=8, min_samples_split=200, n_estimators=200 .........
[CV]  max_features=8, min_samples_split=200, n_estimators=200, total=   0.5s
[CV] max_features=8, min_samples_split=200, n_estimators=250 .........
[CV]  max_features=8, min_samples_split=200, n_estimators=250, total=   0.7s
[CV] max_features=8, min_samples_split=200, n_estimators=250 .........
[CV]  max_features=8, min_samples_split=200, n_estimators=250, total=   0.7s
[CV] max_features=8, min_samples_split=200, n_estimators=300 .........
[CV]  max_features=8, min_samples_split=200, n_estimators=300, total=   0.9s
[CV] max_features=8, min_samples_split=200, n_estimators=300 .........
[CV]  max_features=8, min_samples_split=200, n_estimators=300, total=   1.0s
[CV] max_features=8, min_samples_sp

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:   56.8s finished


Best parameters set found on development set:
{'max_features': 8, 'min_samples_split': 200, 'n_estimators': 350}
0.7714285714285715


In [35]:
# Make a set of predictions for the test data
y_pred = my_tuned_model.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
model_test_accuracy_comparisons["Tuned Random Forest"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_test), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.7911111111111111
             precision    recall  f1-score   support

          0       0.75      0.81      0.78       192
          1       0.99      0.87      0.93       185
          2       0.69      0.73      0.71       157
          3       0.66      0.86      0.74       188
          4       0.56      0.63      0.59       157
          5       0.90      0.86      0.88       170
          6       0.70      0.33      0.45       173
          7       0.88      0.83      0.86       181
          8       0.90      0.94      0.92       199
          9       0.89      0.95      0.92       198

avg / total       0.80      0.79      0.78      1800

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,155,0,5,21,1,0,0,0,10,0,192
1,1,161,4,15,4,0,0,0,0,0,185
2,2,0,115,1,29,0,7,0,3,0,157
3,8,2,3,162,9,0,4,0,0,0,188
4,0,0,20,27,99,0,10,0,1,0,157
5,0,0,0,0,0,147,0,16,2,5,170
6,41,0,18,18,36,0,57,0,3,0,173
7,0,0,0,0,0,12,0,151,0,18,181
8,0,0,2,3,0,2,3,0,188,1,199


#### Bagging

Train and evaluate a simple model 

In [38]:
# Do the same job with random forests
my_model = ensemble.BaggingClassifier(base_estimator = tree.DecisionTreeClassifier(criterion="entropy", min_samples_leaf = 50), \
                                      n_estimators=10)
my_model.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=50, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [39]:
# Make a set of predictions for the validation data
y_pred = my_model.predict(X_valid)

# Print performance details
accuracy = metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
# model_valid_accuracy_comparisons["Bagging"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_valid), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.5500459136822773
                        precision    recall  f1-score   support

                carpet       0.41      0.29      0.34        62
              concrete       0.55      0.75      0.64       212
         fine_concrete       0.45      0.45      0.45        99
            hard_tiles       0.00      0.00      0.00         7
hard_tiles_large_space       0.44      0.68      0.53        90
              soft_pvc       0.55      0.65      0.59       209
            soft_tiles       0.60      0.31      0.41        81
                 tiled       0.65      0.63      0.64       149
                  wood       0.76      0.34      0.47       180

           avg / total       0.57      0.55      0.54      1089

Confusion Matrix


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted,carpet,concrete,fine_concrete,hard_tiles_large_space,soft_pvc,soft_tiles,tiled,wood,All
True,,,,,,,,,
carpet,18,17,2,7,0,0,14,4,62
concrete,2,159,16,6,24,1,2,2,212
fine_concrete,0,14,45,17,17,0,3,3,99
hard_tiles,0,0,0,3,4,0,0,0,7
hard_tiles_large_space,1,6,1,61,16,1,2,2,90
soft_pvc,3,34,3,14,135,1,13,6,209
soft_tiles,10,18,9,4,9,25,4,2,81
tiled,3,11,7,6,13,14,94,1,149
wood,7,29,18,22,29,0,13,62,180


Choose parameters using a grid search

In [ ]:
# Set up the parameter grid to seaerch
param_grid = [
 {'n_estimators': list(range(50, 501, 50)),
  'base_estimator': [tree.DecisionTreeClassifier(criterion="entropy", max_depth = 6, min_samples_leaf = 200)]}
]

# Perform the search
my_tuned_model = GridSearchCV(ensemble.BaggingClassifier(), param_grid, cv=cv_folds, verbose = 2)
my_tuned_model.fit(X_train_plus_valid, y_train_plus_valid)

# Print details
print("Best parameters set found on development set:")
print(my_tuned_model.best_params_)
model_tuned_params_list["Tuned Bagging"] = my_tuned_model.best_params_
print(my_tuned_model.best_score_)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=200, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=50 
[CV]  base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=200, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=50, total=   5.6s
[CV] base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=No

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.8s remaining:    0.0s


[CV]  base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=200, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=50, total=   5.3s
[CV] base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=200, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=100 
[CV]  base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_spl

[CV]  base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=200, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=350, total=  39.8s
[CV] base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=200, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), n_estimators=350 


In [ ]:
# Make a set of predictions for the test data
y_pred = my_tuned_model.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
model_test_accuracy_comparisons["Tuned Bagging"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_test), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

#### AdaBoost

Train and evaluate a simple model 

In [20]:
# Do the same job with random forests
my_model = ensemble.AdaBoostClassifier(base_estimator = tree.DecisionTreeClassifier(criterion="entropy", min_samples_leaf = 200), \
                                       n_estimators=10)
my_model.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=200, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=10, random_state=None)

In [21]:
# Make a set of predictions for the validation data
y_pred = my_model.predict(X_valid)

# Print performance details
accuracy = metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
#model_valid_accuracy_comparisons["AdaBoost"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_valid), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.2809917355371901
                        precision    recall  f1-score   support

                carpet       0.24      0.13      0.17        62
              concrete       0.29      0.22      0.25       212
         fine_concrete       0.20      0.28      0.23        99
            hard_tiles       0.50      0.14      0.22         7
hard_tiles_large_space       0.19      0.20      0.19        90
              soft_pvc       0.32      0.44      0.37       209
            soft_tiles       0.29      0.30      0.29        81
                 tiled       0.29      0.26      0.27       149
                  wood       0.32      0.27      0.29       180

           avg / total       0.28      0.28      0.28      1089

Confusion Matrix


Predicted,carpet,concrete,fine_concrete,hard_tiles,hard_tiles_large_space,soft_pvc,soft_tiles,tiled,wood,All
True,,,,,,,,,,
carpet,8,10,8,0,3,9,3,10,11,62
concrete,6,47,27,0,21,52,21,19,19,212
fine_concrete,1,12,28,1,8,30,7,3,9,99
hard_tiles,0,0,0,1,3,0,1,0,2,7
hard_tiles_large_space,0,7,11,0,18,20,7,9,18,90
soft_pvc,1,24,24,0,13,93,5,29,20,209
soft_tiles,1,8,3,0,13,12,24,9,11,81
tiled,3,31,16,0,8,33,8,39,11,149
wood,14,21,23,0,9,39,8,18,48,180


Choose parameters using a grid search

In [ ]:
# Set up the parameter grid to seaerch
param_grid = [
 {'n_estimators': list(range(50, 501, 50)),
 'base_estimator': [tree.DecisionTreeClassifier(criterion="entropy", max_depth = 6, min_samples_leaf = 200)]}
]

# Perform the search
my_tuned_model = GridSearchCV(ensemble.AdaBoostClassifier(), param_grid, cv=cv_folds, verbose = 2)
my_tuned_model.fit(X_train_plus_valid, y_train_plus_valid)

# Print details
print("Best parameters set found on development set:")
print(my_tuned_model.best_params_)
model_tuned_params_list["Tuned AdaBoost"] = my_tuned_model.best_params_
print(my_tuned_model.best_score_)

In [ ]:
# Make a set of predictions for the test data
y_pred = my_tuned_model.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
model_test_accuracy_comparisons["Tuned AdaBoost"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_test), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

#### Logistic Regression

Train and evaluate a simple model 

In [15]:
# Do the same job with logistic regression
my_model = linear_model.LogisticRegression()
my_model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
# Make a set of predictions for the test data
y_pred = my_model.predict(X_valid)

# Print performance details
accuracy = metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
#model_valid_accuracy_comparisons["Logistic Regression"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_valid), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.3002754820936639
                        precision    recall  f1-score   support

                carpet       0.00      0.00      0.00        62
              concrete       0.26      0.63      0.37       212
         fine_concrete       0.00      0.00      0.00        99
            hard_tiles       0.00      0.00      0.00         7
hard_tiles_large_space       0.00      0.00      0.00        90
              soft_pvc       0.30      0.61      0.40       209
            soft_tiles       0.00      0.00      0.00        81
                 tiled       0.00      0.00      0.00       149
                  wood       0.47      0.36      0.41       180

           avg / total       0.19      0.30      0.22      1089

Confusion Matrix


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted,concrete,soft_pvc,wood,All
True,,,,
carpet,45,14,3,62
concrete,134,70,8,212
fine_concrete,52,31,16,99
hard_tiles,0,6,1,7
hard_tiles_large_space,31,54,5,90
soft_pvc,72,128,9,209
soft_tiles,25,54,2,81
tiled,94,26,29,149
wood,68,47,65,180


Choose parameters using a grid search

In [ ]:
# Set up the parameter grid to seaerch
param_grid = [
 {'multi_class': ['ovr'], 
 'C': [x / 10.0 for x in range(2, 21, 2)],
 'solver':['liblinear'],
  'max_iter':[1000]}
]

# Perform the search
my_tuned_model = GridSearchCV(linear_model.LogisticRegression(), param_grid, cv=cv_folds, verbose = 2)
my_tuned_model.fit(X_train_plus_valid, y_train_plus_valid)

# Print details
print("Best parameters set found on development set:")
print(my_tuned_model.best_params_)
model_tuned_params_list["Tuned Logistic Regression"] = my_tuned_model.best_params_
print(my_tuned_model.best_score_)


In [ ]:
# Make a set of predictions for the test data
y_pred = my_tuned_model.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
model_test_accuracy_comparisons["Tuned Logistic Regression"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_test), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

#### Nearest Neighbour

Train and evaluate a simple model 

In [ ]:
# Do the same job with random forests
my_model = neighbors.KNeighborsClassifier()
my_model = my_model.fit(X_train,y_train)

In [ ]:
# Make a set of predictions for the test data
y_pred = my_model.predict(X_valid)

# Print performance details
accuracy = metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
model_valid_accuracy_comparisons["kNN"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_valid), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Choose parameters using a grid search

In [ ]:
# Set up the parameter grid to seaerch
param_grid = [
               {'n_neighbors': list(range(1, 50, 5))}
]

# Perform the search
my_tuned_model = GridSearchCV(neighbors.KNeighborsClassifier(), param_grid, cv=cv_folds, verbose = 2)
my_tuned_model.fit(X_train_plus_valid, y_train_plus_valid)

# Print details
print("Best parameters set found on development set:")
print(my_tuned_model.best_params_)
model_tuned_params_list["Tuned kNN"] = my_tuned_model.best_params_
print(my_tuned_model.best_score_)

In [ ]:
# Make a set of predictions for the test data
y_pred = my_tuned_model.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
model_test_accuracy_comparisons["Tuned kNN"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_test), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

### Multi Layer Perceptron

Train and evaluate a simple model 

In [43]:
# Do the same job with random forests
my_model = neural_network.MLPClassifier(hidden_layer_sizes=(10000, 200))
my_model = my_model.fit(X_train,y_train)

In [58]:
# Make a set of predictions for the test data
y_pred = my_model.predict(X_valid)

# y_pred[-7:] = 'hard_tiles_large_space'
# print(y_pred[-7:], len(y_pred))

# Print performance details
accuracy = metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
#model_valid_accuracy_comparisons["MLP"] = accuracy
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_valid), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.6097337006427915
                        precision    recall  f1-score   support

                carpet       0.49      0.45      0.47        62
              concrete       0.64      0.51      0.57       212
         fine_concrete       0.44      0.56      0.49        99
            hard_tiles       0.00      0.00      0.00         7
hard_tiles_large_space       0.53      0.63      0.58        90
              soft_pvc       0.62      0.88      0.73       209
            soft_tiles       0.59      0.59      0.59        81
                 tiled       0.77      0.66      0.71       149
                  wood       0.71      0.48      0.57       180

           avg / total       0.62      0.61      0.60      1089

Confusion Matrix


Predicted,carpet,concrete,fine_concrete,hard_tiles,hard_tiles_large_space,soft_pvc,soft_tiles,tiled,wood,All
True,,,,,,,,,,
carpet,28,7,1,0,4,1,4,9,8,62
concrete,7,108,26,0,18,30,9,5,9,212
fine_concrete,0,11,55,1,8,13,2,5,4,99
hard_tiles,0,0,0,0,2,2,0,0,3,7
hard_tiles_large_space,3,12,3,0,57,6,5,0,4,90
soft_pvc,0,6,7,0,4,184,0,6,2,209
soft_tiles,4,4,4,0,4,10,48,5,2,81
tiled,4,10,6,1,2,18,7,98,3,149
wood,11,10,23,0,9,34,7,0,86,180


Choose parameters using a grid search

In [ ]:
# Set up the parameter grid to seaerch
param_grid = [
               {'hidden_layer_sizes': [(400), (400, 200), (400, 200, 100)], 
               'alpha': list(10.0 ** -np.arange(1, 7))}
]

# Perform the search
my_tuned_model = GridSearchCV(neural_network.MLPClassifier(), param_grid, cv=cv_folds, verbose = 2)
my_tuned_model.fit(X_train_plus_valid, y_train_plus_valid)

# Print details
print("Best parameters set found on development set:")
print(my_tuned_model.best_params_)
model_tuned_params_list["Tuned MLP"] = my_tuned_model.best_params_
print(my_tuned_model.best_score_)

In [40]:
# Make a set of predictions for the test data
y_pred = my_model.predict(X_test)
print(len(y_pred))

#idx = pd.Int64Index(y_pred)
df_op = pd.DataFrame(data =(y_pred))
#print(df_op)
df_op['series_id'] = range(0, len(df_op))

df_op.columns = ['surface', 'series_id']

columnsTitles=["series_id", "surface"]
df_op=df_op.reindex(columns=columnsTitles)
print(df_op)

df_op.to_csv('sub3.csv', sep=',', index=False)

# Print performance details
#accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
#model_test_accuracy_comparisons["Tuned MLP"] = accuracy
#print("Accuracy: " +  str(accuracy))
#print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
#print("Confusion Matrix")
#pd.crosstab(np.array(y_test), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

3816
      series_id                 surface
0             0  hard_tiles_large_space
1             1                   tiled
2             2                   tiled
3             3                  carpet
4             4                   tiled
5             5                concrete
6             6                concrete
7             7                    wood
8             8                concrete
9             9                  carpet
10           10                concrete
11           11                soft_pvc
12           12  hard_tiles_large_space
13           13                concrete
14           14                soft_pvc
15           15                concrete
16           16                concrete
17           17           fine_concrete
18           18                    wood
19           19                soft_pvc
20           20                  carpet
21           21                concrete
22           22                concrete
23           23  hard_tiles_large_s

### Compare Results

In [ ]:
display(model_test_accuracy_comparisons)

In [ ]:
plt.xlim(0, 1.0)
_ = plt.barh(range(len(model_test_accuracy_comparisons)), list(model_test_accuracy_comparisons.values()), align='center')
_ = plt.yticks(range(len(model_test_accuracy_comparisons)), list(model_test_accuracy_comparisons.keys()))

In [ ]:
display(model_valid_accuracy_comparisons)

In [ ]:
plt.xlim(0, 1.0)
_ = plt.barh(range(len(model_valid_accuracy_comparisons)), list(model_valid_accuracy_comparisons.values()), align='center')
_= plt.yticks(range(len(model_valid_accuracy_comparisons)), list(model_valid_accuracy_comparisons.keys()))

In [ ]:
display(model_tuned_params_list)

### Test Best Model On Test Dataset

In [ ]:
test_dataset = pd.read_csv('fashion-mnist_test.csv')
test_dataset.head()

In [ ]:
test_X = test_dataset[test_dataset.columns[1:]]
test_Y = np.array(test_dataset["label"])

In [ ]:
test_X = test_X/255

In [26]:
my_model = linear_model.LogisticRegression(C=0.4,max_iter = 1000,multi_class='ovr',solver='liblinear')
my_model = my_model.fit(X,Y)

In [31]:
# Make a set of predictions for the valid data
y_pred = my_model.predict(X_valid)

# Print performance details
accuracy = metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_valid), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.3039485766758494
                        precision    recall  f1-score   support

                carpet       0.00      0.00      0.00        62
              concrete       0.27      0.64      0.38       212
         fine_concrete       0.00      0.00      0.00        99
            hard_tiles       0.00      0.00      0.00         7
hard_tiles_large_space       0.00      0.00      0.00        90
              soft_pvc       0.30      0.61      0.40       209
            soft_tiles       0.00      0.00      0.00        81
                 tiled       0.00      0.00      0.00       149
                  wood       0.46      0.37      0.41       180

           avg / total       0.18      0.30      0.22      1089

Confusion Matrix


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Predicted,concrete,soft_pvc,wood,All
True,,,,
carpet,45,14,3,62
concrete,136,69,7,212
fine_concrete,54,32,13,99
hard_tiles,0,6,1,7
hard_tiles_large_space,31,53,6,90
soft_pvc,68,128,13,209
soft_tiles,23,56,2,81
tiled,89,26,34,149
wood,66,47,67,180
